Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

Создал датасет со столбцами температурв в градусах цельсия, температура в градусах кельвина, день месяца, давление в бар, давление в миллиметрах ртутного столба и произведение даления в бар и температуры в градусах цельсия

In [14]:
# Создаём сэмпл
n_samples = 1000

t_celsius = np.random.choice(40, n_samples) + 5
p_bar = np.random.choice(100, n_samples) + 1
day = np.random.choice(30, n_samples) + 1
t_kelvin = t_celsius + 273
p_mercury_column = p_bar + 750

param = 11 * p_bar * t_celsius + 27

data = pd.DataFrame({'t_celsius': t_celsius, 'p_bar': p_bar, 'day': day, 't_kelvin': t_kelvin,
                     'p_mercury_column': p_mercury_column, 'param': param})
data.head(5)

,t_celsius,p_bar,day,t_kelvin,p_mercury_column,param
0,15,58,14,288,808,9597
1,25,60,13,298,810,16527
2,10,87,24,283,837,9597
3,20,84,5,293,834,18507
4,43,68,29,316,818,32191


Обучаем нашу модель. Находим среднюю абсолютную ошибку, а также коэфициенты линейной модели

In [15]:
from sklearn.metrics import mean_absolute_error

X = data[['t_celsius', 'p_bar', 'day', 't_kelvin', 'p_mercury_column']]
y = data['param']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['t_celsius', 'p_bar', 'day', 't_kelvin', 'p_mercury_column']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 6.08186682e+14 -3.36284940e+14  3.52734375e+00 -6.08186682e+14
  3.36284940e+14]
Bias: -8.617874116573029e+16
Error: 2602.991


Ошибка получилось большой, а коэфициенты очень маленькими

In [16]:
y.median()

9817.0

In [ ]:
Оставим только два параметра для обучения

In [17]:
from sklearn.metrics import mean_absolute_error

X = data[['t_celsius', 'p_bar']]
y = data['param']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['t_celsius', 'p_bar']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [534.22905267 267.55412564]
Bias: -12881.494173066114
Error: 2602.8808654987533


Ошибка по сравнению с предыдущим обучением не изменилась

Создал два новых признака произведение температуры в градусах и давления в бар.
А также произведение температуры в кельвинах и давления в мм рт столба

In [18]:
# Создаем новый признак
data['mult1'] = data['t_celsius'] * data['p_bar']
data['mult2'] = data['t_kelvin'] * data['p_mercury_column']
data.head(5)

,t_celsius,p_bar,day,t_kelvin,p_mercury_column,param,mult1,mult2
0,15,58,14,288,808,9597,870,232704
1,25,60,13,298,810,16527,1500,241380
2,10,87,24,283,837,9597,870,236871
3,20,84,5,293,834,18507,1680,244362
4,43,68,29,316,818,32191,2924,258488


In [ ]:
Сперва попробуем обучить модель на признаке mult2

In [20]:
X = data[['mult2']]
y = data['param']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult1']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.77981532]
Bias: -172026.4075641105
Error: 184078.95720518474


Хотя давления в барах и мм рт столба коррелируются между собой на 1, так же как и температуры между собой ошибка получилась даже больше чем в других моделях

In [ ]:
Обучим модель на признаке mult1

In [19]:
X = data[['mult1']]
y = data['param']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult1']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [11.]
Bias: 26.999999999994543
Error: 3.795889824687038e-12


param = 11 * p_bar * t_celsius + 27
Видим, что модель подобрала правильно коэфициенты, а ошибка очень мала

Добавил параметр день к признаку mult1

In [21]:
X = data[['day', 'mult1']]
y = data['param']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['day', 'mult1']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-1.60423817e-15  1.10000000e+01]
Bias: 26.999999999985448
Error: 9.465736638958333e-12


In [ ]:
Ошибка тоже получилась очень маленькой. BIAS не изменился и найден верно. Параметр 'день' не сильно помешал обучению